In [1]:
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(42)

Implenting GoogLeNet from the Inception Paper by Google


In [84]:
def inception_module(prev_layer,filters):
    """
    filters prarm order[3x3reduce,5x5reduce,1x1,3x3,5x5,prodj]
    """
    conv_1 = keras.layers.Conv2D(filters=filters[0],kernel_size=1,
                                padding='same',activation='relu')(prev_layer)
    conv_2 = keras.layers.Conv2D(filters=filters[1],kernel_size=1,
                                padding='same',activation='relu')(prev_layer)
    conv_a = keras.layers.Conv2D(filters=filters[2],kernel_size=1,
                                padding='same',activation='relu')(prev_layer)
    conv_b = keras.layers.Conv2D(filters=filters[3],kernel_size=3,
                                padding='same',activation='relu')(conv_1)
    conv_c = keras.layers.Conv2D(filters=filters[4],kernel_size=5,
                                strides=1,padding='same',activation='relu')(conv_2)
    pooling = keras.layers.MaxPooling2D((3,3),strides=1,padding='same')(prev_layer)
    conv_3 = keras.layers.Conv2D(filters=filters[5],kernel_size=1,
                                 padding='same',activation='relu')(pooling)

    concat = tf.keras.layers.Concatenate(axis=3)([conv_a,conv_b,conv_c,conv_3])
    
    return concat

In [94]:
# Section A 

input_layer = keras.layers.Input(shape=[224,224,3])
base_conv = keras.layers.Conv2D(filters=64,kernel_size=7,strides=2,
                                padding='same',activation='relu')(input_layer)
pooling = keras.layers.MaxPooling2D((3,3),strides=2,padding='same')(base_conv)
lnorm1 = keras.layers.LayerNormalization()(pooling)
conv2 = keras.layers.Conv2D(filters=64,kernel_size=1,strides=1,
                                padding='same',activation='relu')(lnorm1)
conv3 = keras.layers.Conv2D(filters=192,kernel_size=3,strides=1,
                                padding='same',activation='relu')(conv2)
lnorm2 = keras.layers.LayerNormalization()(conv3)
pooling2 = keras.layers.MaxPooling2D((3,3),strides=2, padding='same')(lnorm2)                                      
#incept_a = inception_module(base_conv,[128,128,32])


In [95]:
# Section B
x = inception_module(pooling2,[96,16,64,128,32,32])
x = inception_module(x,[128,32,128,192,96,64])
pooling3 = keras.layers.MaxPooling2D((3,3),strides=2, padding='same')(x)
x = inception_module(pooling3,[96,16,192,208,48,64])
x = inception_module(x,[112,24,160,224,64,64])
x = inception_module(x,[128,24,128,256,64,64])
x = inception_module(x,[144,32,112,288,64,64])
x = inception_module(x,[160,32,256,320,128,128])
pooling4 = keras.layers.MaxPooling2D((3,3),strides=2, padding='same')(x)
x = inception_module(pooling4,[160,32,256,320,128,128])
x = inception_module(x,[192,48,384,384,128,128])

In [96]:
# Section C
global_pool = keras.layers.AveragePooling2D(pool_size=7,strides=1,padding='valid')(x)
drp = keras.layers.Dropout(0.4)(global_pool)
out = keras.layers.Dense(units=10,activation='softmax')(drp)

In [97]:
googlenet_model = keras.models.Model(inputs=[input_layer],outputs=[out])

In [98]:
googlenet_model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_292 (Conv2D)            (None, 112, 112, 64  9472        ['input_27[0][0]']               
                                )                                                                 
                                                                                                  
 max_pooling2d_72 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv2d_292[0][0]']             
 )                                                                                         

In [100]:
import math

epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lr_schedule = keras.callbacks.LearningRateScheduler(decay, verbose=1)
sgd = keras.optimizers.SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)
googlenet_model.compile(loss='categorical_crossentropy', optimizer=sgd,
                        metrics=['accuracy'])
